In [3]:
import math
import numpy as np
import pandas as pd
import os
import glob

from pykrx import stock
from pykrx import bond

import time
from time import sleep
from datetime import datetime
from datetime import timedelta

from pyarrow import csv
import pyarrow as pa
import pyarrow.parquet as pq

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import psycopg2 as pg2
from sqlalchemy import create_engine

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage


import warnings
warnings.filterwarnings('ignore')

# 경로 변경
os.chdir('/home/shjj08choi4/finance_mlops')


# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("key_value/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# 빅쿼리 정보
project_id = 'owenchoi-404302'
dataset_id = 'finance_mlops'


# BigQuery 클라이언트 객체 생성
client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)


# GCP 클라이언트 객체 생성
storage_client = storage.Client(credentials = credentials,
                         project = credentials.project_id)
bucket_name = 'finance-mlops-proj'    # 서비스 계정 생성한 bucket 이름 입력


now = datetime.now()
# now = now + timedelta(days=-2)
today_date1 = now.strftime('%Y%m%d')
today_date2 = now.strftime('%Y-%m-%d')
today_date_time_csv = now.strftime("%Y%m%d_%H%M")

today_date1 = '2024010'
today_date2 = '2023-01-07'

today_date1 = 'reset'

In [2]:
glob.glob("data_crawler/cleaning/kor_stock_ohlcv/*")

['data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_kospi.parquet',
 'data_crawler/cleaning/kor_stock_ohlcv/buy_sell_count_kosdaq.parquet',
 'data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_kosdaq.parquet',
 'data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_anal_kosdaq.parquet',
 'data_crawler/cleaning/kor_stock_ohlcv/buy_sell_count_kospi.parquet',
 'data_crawler/cleaning/kor_stock_ohlcv/kor_stock_ohlcv_anal_kospi.parquet']

In [4]:

data_crawler_dashboard_gemini_main_view.parquet

t = pq.read_table('data_crawler/dashboard/gemini_main_view.parquet')
df_raw_total_2_KOSPI_reset = t.to_pandas()

In [6]:
df_raw_total_2_KOSPI_reset['response_msg']

0    **코스피 증권 보고서**\n\n**일자:** 2024년 11월 1일\n\n**종목...
Name: response_msg, dtype: object

In [ ]:

t = pq.read_table('data_crawler/dashboard/gemini_main_view.parquet')
df_raw_total_2_KOSPI_reset = t.to_pandas()

In [7]:
t = pq.read_table('data_crawler/cleaning/kor_stock_ohlcv/df_raw_total_2_KOSPI_reset.parquet')
df_raw_total_2_KOSPI_reset = t.to_pandas()

In [9]:
df_raw_total_2_KOSPI_reset.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume',
       'price_change_percentage', 'ticker', 'corp_name', 'market', 'MA5',
       'MA20', 'MA60', 'MA120', 'upper', 'lower', 'MACD_DIFF', 'MACD',
       'MACD_Signal', '변화량', '상승폭', '하락폭', 'AU', 'AD', 'RSI', 'MA5-20',
       'MA20-60', 'MA60-120'],
      dtype='object')

In [19]:
df_raw_total_2_KOSPI_reset_222 = df_raw_total_2_KOSPI_reset[['date', 'open', 'high', 'low', 'close', 'volume',
                            'ticker', 'corp_name', 'market', 
                            'MA5','MA20', 'MA60', 'MA120', 
                            'upper', 'lower', 'MACD_DIFF', 'MACD','MACD_Signal', 'RSI']]

df_raw_total_2_KOSPI_reset_222 = df_raw_total_2_KOSPI_reset_222[df_raw_total_2_KOSPI_reset_222['market'] == 'KOSPI'].reset_index(drop = True)

In [22]:
df_raw_total_2_KOSPI_reset_222[df_raw_total_2_KOSPI_reset_222['ticker'] == '017860']

,date,open,high,low,close,volume,ticker,corp_name,market,MA5,MA20,MA60,MA120,upper,lower,MACD_DIFF,MACD,MACD_Signal,RSI
235177,2023-12-22 00:00:00+00:00,370000,400000,350500,400000.0,1040171,017860,DS단석,KOSPI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235178,2023-12-26 00:00:00+00:00,471000,495000,318500,335000.0,4181604,017860,DS단석,KOSPI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235179,2023-12-27 00:00:00+00:00,366500,372000,270500,282000.0,1731242,017860,DS단석,KOSPI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235180,2023-12-28 00:00:00+00:00,278500,299000,262000,289000.0,1018475,017860,DS단석,KOSPI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235181,2024-01-02 00:00:00+00:00,278500,294000,278000,279000.0,304472,017860,DS단석,KOSPI,317000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235182,2024-01-03 00:00:00+00:00,274000,303000,272000,298500.0,652663,017860,DS단석,KOSPI,296700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235183,2024-01-04 00:00:00+00:00,294500,300500,279000,280500.0,273558,017860,DS단석,KOSPI,285800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235184,2024-01-05 00:00:00+00:00,278500,282500,260000,261500.0,284726,017860,DS단석,KOSPI,281700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
table_from_pandas = pa.Table.from_pandas(df_raw_total_2_KOSPI_reset_222,preserve_index = False)
pq.write_table(table_from_pandas, f'data_crawler/cleaning/kor_stock_ohlcv/df_raw_total_2_KOSPI_{today_date1}_2.parquet')
